1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

2. Import phi and phi_test from train and test datasets using NumPy's loadtxt function

In [11]:
phi = np.loadtxt('train.csv', skiprows=1, delimiter=',', dtype='float', usecols=tuple(range(1, 14)))
phi_test = np.loadtxt('test.csv', skiprows=1, delimiter=',', dtype='float', usecols=tuple(range(1, 14)))
phi[0], phi_test[0]

(array([  0.95577,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.047  ,
         88.8    ,   4.4534 ,   4.     , 307.     ,  21.     , 306.38   ,
         17.28   ]),
 array([1.0612e-01, 3.0000e+01, 4.9300e+00, 0.0000e+00, 4.2800e-01,
        6.0950e+00, 6.5100e+01, 6.3361e+00, 6.0000e+00, 3.0000e+02,
        1.6600e+01, 3.9462e+02, 1.2400e+01]))

3. Import y from train dataset using the loadtxt function

In [17]:
y = np.loadtxt('train.csv', skiprows=1, dtype='float', usecols=14, ndmin=2, delimiter=',')
y[0]

array([14.8])

4. Concatenate coloumn of 1s to right of phi and phi_test

In [22]:
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)
phi[0], phi_test[0]

(array([  0.95577,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.047  ,
         88.8    ,   4.4534 ,   4.     , 307.     ,  21.     , 306.38   ,
         17.28   ,   1.     ]),
 array([1.0612e-01, 3.0000e+01, 4.9300e+00, 0.0000e+00, 4.2800e-01,
        6.0950e+00, 6.5100e+01, 6.3361e+00, 6.0000e+00, 3.0000e+02,
        1.6600e+01, 3.9462e+02, 1.2400e+01, 1.0000e+00]))

5. Apply min max scaling on each coloumn of phi and phi_test
> ![](2022-10-27-18-02-57.png)

In [26]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(phi)
phi = scaler.transform(phi)
phi_test = scaler.transform(phi_test)

6. Apply log scaling on y

In [27]:
y = np.log(y)

7. Define a function to calculate change in error function based on phi, w and p norm
> ![](2022-10-27-18-07-08.png)

In [28]:
def CostFunction(p, beta, phi, lamb):
    if p == 2:
        dJ = (2 * (np.dot(np.dot(np.transpose(phi), phi), beta) - np.dot(np.transpose(phi), y)) + 
            lamb*p*np.power(np.absolute(beta), (p-1)))
    elif p > 1 and p < 2:
        dJ = (2 * (np.dot(np.dot(np.transpose(phi), phi), beta) - np.dot(np.transpose(phi), y)) + 
            lamb*p*np.power(np.absolute(beta), (p-1))*np.sign(beta))
    return dJ

8. Make a dictionary containing filenames as keys and p as values

In [29]:
# I would like to append predicted MEDV values in output.csv for different p values

filenames = {'output.csv':[2.0, 1.75, 1.5, 1.3, 1.0]}

9. For each item in this dictionary
    + Set the `w` to all 0s
    + Set an appropriate value for `lambda` and `step size`
    + Calculate new value of `w`
    + Repeat steps until error between consecutive `w`s is less than threshold
    + Load values of `id` from test data file
    + Calculate `y` for test data using `phi_test` and applying inverse log
    + Save the `id`s and `y` according to filename from dictionary